In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import  ParameterGrid, StratifiedKFold
from sklearn.metrics import log_loss ,roc_auc_score,roc_curve,auc
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
%config IPCompleter.greedy=True

def gini(y,pred):
    fpr,tpr,thr = roc_curve(y,pred,pos_label=1)
    g = 1 - 2 * auc(fpr,tpr)
    return g

In [2]:
df = pd.read_csv('train.csv')

#データ型確認
df.dtypes

id                  int64
target              int64
ps_ind_01           int64
ps_ind_02_cat       int64
ps_ind_03           int64
ps_ind_04_cat       int64
ps_ind_05_cat       int64
ps_ind_06_bin       int64
ps_ind_07_bin       int64
ps_ind_08_bin       int64
ps_ind_09_bin       int64
ps_ind_10_bin       int64
ps_ind_11_bin       int64
ps_ind_12_bin       int64
ps_ind_13_bin       int64
ps_ind_14           int64
ps_ind_15           int64
ps_ind_16_bin       int64
ps_ind_17_bin       int64
ps_ind_18_bin       int64
ps_reg_01         float64
ps_reg_02         float64
ps_reg_03         float64
ps_car_01_cat       int64
ps_car_02_cat       int64
ps_car_03_cat       int64
ps_car_04_cat       int64
ps_car_05_cat       int64
ps_car_06_cat       int64
ps_car_07_cat       int64
ps_car_08_cat       int64
ps_car_09_cat       int64
ps_car_10_cat       int64
ps_car_11_cat       int64
ps_car_11           int64
ps_car_12         float64
ps_car_13         float64
ps_car_14         float64
ps_car_15   

In [3]:
#特徴稜確認
df.describe() 

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,5.952120e+05,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,7.438036e+05,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,4.293678e+05,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000e+00,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.719915e+05,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.435475e+05,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.115549e+06,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.488027e+06,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
#出力は595212*59
df.shape 

(595212, 59)

In [5]:
x_train = df.drop('target',axis=1)
y_train = df['target'].values

In [6]:
# param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
param_range = [0.1]
all_params = {'C':param_range, 
              'kernel':['linear','rbf'],
              'gamma':param_range
             }

scaler = StandardScaler()
scaler.fit(x_train)
scaler.transform(x_train)

array([[-1.7323077 ,  0.0502179 ,  0.9645855 , ..., -0.63472992,
        -0.73222528,  2.34997253],
       [-1.73230305, -0.45386836, -0.54009356, ..., -0.63472992,
         1.36569991, -0.42553689],
       [-1.73229373,  1.56247669,  3.9739436 , ..., -0.63472992,
         1.36569991, -0.42553689],
       ...,
       [ 1.73327863, -0.45386836, -0.54009356, ..., -0.63472992,
        -0.73222528, -0.42553689],
       [ 1.73328794,  1.56247669,  0.9645855 , ...,  1.57547323,
        -0.73222528, -0.42553689],
       [ 1.73330192, -0.95795462, -0.54009356, ..., -0.63472992,
        -0.73222528, -0.42553689]])

In [ ]:
skf = StratifiedKFold(n_splits=2, random_state=0, shuffle=True)

min_score = 100
min_params = None

for params in tqdm(ParameterGrid(all_params)):
    gini_scores = []
    
    for train_index, val_index in skf.split(x_train, y_train):
        train_x = x_train.iloc[train_index]
        train_y = y_train[train_index]
        val_x = x_train.iloc[val_index]
        val_y = y_train[val_index]
        
        clf = SVC(**params)
        clf.fit(train_x, train_y)
        pred = clf.predict_proba(val_x)[:,1]
        
        gini_score = gini(val_y, pred)
        gini_scores.append(gini_score)
        
    total_score = np.mean(gini_scores)
    
    if total_score <= min_score:
        min_score = total_score
        min_params = params
        
clf = SVC(**min_params)
clf.fit(x_train, y_train)    

  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
df_test = pd.read_csv('test.csv')
pred_test = clf.predict_proba(df_test)[:,1]
df_sample = pd.read('sample.csv')
df_sample['target'] = pred_test

df_sample.to_csv('submmit.csv', index=False)


,id,target
0,0,0.0364
1,1,0.0364
2,2,0.0364
3,3,0.0364
4,4,0.0364
...,...,...
892811,1488022,0.0364
892812,1488023,0.0364
892813,1488024,0.0364
892814,1488025,0.0364
